## WHY DOESN'T THIS WORK??:


In [6]:
class Try
  attr_accessor :geneID
  def initialize (params = {}) 
    geneID = params.fetch(:geneID, "no AGI code defined")
    @geneID = geneID
  end
end

my_file = File.new("/Users/sara/BioCompu/Programming_challenges/Task2/ArabidopsisSubNetwork_GeneList.txt","r")
my_file.collect { |codes|
  object_name = "name_#{codes.to_s}" 
  object_name = Try.new(geneID: codes.strip.to_s)
  #puts "#{object_name} = Try.new(geneID: #{codes.strip})"
  }
puts ""

# TASK

Find protein interactions and:
- Create network object
- Annotate interaction with KEGG and GO
- Create report of which genes interact with which

# MAIN

In [6]:
#----------------------- LOAD REQUIREMENTS ----------------------------
require 'rest-client'
require 'json' # to handle JSON format

true

In [7]:
##------------------- Mark's fuction to prevent web accessing errors --------------------
def fetch(url, headers = {accept: "*/*"}, user = "", pass="")
  response = RestClient::Request.execute({
    method: :get,
    url: url.to_s,
    user: user,
    password: pass,
    headers: headers})
  return response
  
  rescue RestClient::ExceptionWithResponse => e
    $stderr.puts e.response
    response = false
    return response  # now we are returning 'False', and we will check that with an \"if\" statement in our main code
  rescue RestClient::Exception => e
    $stderr.puts e.response
    response = false
    return response  # now we are returning 'False', and we will check that with an \"if\" statement in our main code
  rescue Exception => e
    $stderr.puts e
    response = false
    return response  # now we are returning 'False', and we will check that with an \"if\" statement in our main code
end 

:fetch

In [8]:
#----------------------- CREATE GENE CLASS ----------------------------
class Gene
  attr_accessor :gene_id # AGI gene ID code
  attr_accessor :prot_id # UniProt IDs
  attr_accessor :kegg # Array with KEGG's ID and pathway name
  attr_accessor :go # Array with GO's ID and term name
  attr_accessor :interactions # Array with interactions
  @@all_genes = Array.new
  
  
  def initialize (params = {}) 
##    agi_code = params.fetch(:agi, "AT0G00000")
##    if (/A[Tt]\d[Gg]\d\d\d\d\d/).match(agi_code) #regular expresion for gene identifier format
##      @gene_id = agi_code
##    else 
##      raise "\n Error: invalid gene ID format \n Make sure that ALL gene IDs have AT0G00000 format"
##    end
    @gene_id = params.fetch(:gene_id, "AT0G00000")
    @prot_id = params.fetch(:prot_id, Array.new) #UniProt IDs
    @kegg = params.fetch(:kegg, Array.new) #Array with KEGG's ID and pathway name
    @go = params.fetch(:go, Array.new) #Array with GO's ID and term name
    @interactions = params.fetch(:interactions, Array.new) #Array with GO's ID and term name
    
    @@all_genes = @@all_genes.push(@gene_id.upcase)
    
  end
  
  def show_all_genes
    return @@all_genes
  end
  
  def get_info
    puts "Gene's ID : #{self.gene_id}"
    puts "Proteins's UniProt name: #{self.prot_id}"
    puts "KEGG's ID and pathway name: #{self.kegg}"
    puts "GO's ID and term name: #{self.go}"
    puts "Protein interactions: #{self.interactions}"
  end
  
end

#------------------- LOAD INFO FROM GENE FILE --------------------

#Now I open the gene file and create some arrays that will help create the Class objects
gene_file = File.new("/Users/sara/BioCompu/Programming_challenges/Task2/ArabidopsisSubNetwork_GeneList.txt","r")

genes_list=Array.new 
gene_file.each do |agi| #iterate over the elements of gene_file
  gene_agi_code = agi.strip
  gene_agi_code.upcase!
  genes_list.push(gene_agi_code) #add AGI codes to a list
end



AT4g27030
AT5g54270
AT1g21400
AT5g19120
AT2g13360
AT4g05180
AT1g22690
AT2g45170
AT4g09650
AT5g55620
AT1g31330
AT1g80440
AT3g28270
AT4g12800
AT5g04140
AT5g64040
AT1g29660
AT1g15820
AT1g64500
AT1g03130
AT2g20670
AT4g17090
AT1g23740
AT2g46340
AT4g17460
AT1g07010
AT1g32220
AT2g03750
AT3g47470
AT4g12830
AT5g05690
AT5g64410
AT2g21330
AT1g15980
AT2g01890
AT1g12250
AT2g21210
AT4g28660
AT1g52230
AT3g18890
AT4g22890
AT1g11850
AT1g37130
AT2g04039
AT3g48420
AT4g19170
AT5g07020
AT5g65010
AT3g01500
AT1g25230
AT2g34430
AT1g18060
AT2g30600
AT4g33010
AT1g65490
AT3g21670
AT4g27260
AT1g12780
AT1g55480
AT2g17880
AT3g49160
AT4g21210
AT5g14740
AT4g37980
AT1g29395
AT2g39730
AT1g20020
AT2g34420
AT4g37800
AT1g70820
AT3g26740
AT4g28780
AT1g12900
AT1g64680
AT2g25200
AT3g55800
AT4g21280
AT5g15850
AT1g06680
AT2g43550
AT3g02730
AT1g42970
AT2g39470
AT5g17300
AT1g77490
AT3g27690
AT5g01015
AT1g14150
AT1g64720
AT2g26080
AT3g56940
AT4g28030
AT5g20250
AT1g54040
AT3g21870
AT3g48360
AT1g61520
AT2g43010
AT5g22920
AT1g78370


# Web access

In [157]:
#------------------- get protein ID --------------------
def get_protID(my_gene)
  address="http://togows.org/entry/ebi-uniprot/#{my_gene}/accessions.json"
  response = fetch(address)  
  if response  # if there is a response to calling that URI
    body = response.body  # get the "body" of the response
    data = JSON.parse(response.body)
    return data[0]
  end
end


:get_protID

In [158]:
#------------------- get KEGG annotations --------------------
def get_KEGG(my_gene)
  address = "http://togows.org/entry/kegg-genes/ath:#{my_gene}/pathways.json"
  my_KEGG_list=Array.new
  response = fetch(address)  
  if response  # if there is a response to calling that URI
    body = response.body  # get the "body" of the response
    data = JSON.parse(response.body)
    my_KEGG_list.push(data[0].to_a)
    if data[0].length != 0
      my_KEGG_list = data[0].to_a
    end
    return my_KEGG_list
  else 
    return Array.new
  end
end

:get_KEGG

In [159]:
#------------------- get GO annotations --------------------
def get_GO(my_gene)
  address = "http://togows.dbcls.jp/entry/uniprot/#{my_gene}/dr.json"
  my_GO_list=Array.new
  response = fetch(address)  
  if response  # if there is a response to calling that URI
    body = response.body  # get the "body" of the response
    data = JSON.parse(response.body)
    if data[0]["GO"]
      data[0]["GO"].each do |go| 
      if go[1] =~ /P:/#if its a biological process it will have the key 'P'
        my_GO_list.push(go[0..1])
      end
    end
      return my_GO_list
      puts ""
    end
  else 
    return Array.new
  end
end

:get_GO

In [9]:
def get_interactions(my_gene,all_genes)
  address="http://www.ebi.ac.uk/Tools/webservices/psicquic/intact/webservices/current/search/query/#{my_gene}?format=tab25"
  response = fetch(address)
  interactions_array=Array.new
  #puts "get_interactions function has been called"
  
  if response  # if there is a response to calling that URI
    body = response.body  # get the "body" of the response
    data = body.split("\n").to_a #first I split the rows into different arrays
    (0..data.length-1).each do |i|

      data[i] = data[i].split("\t") #then I create an array's element per each tab separated value 
      data[i] = data[i][4..5] #these columns of the tab25 format cointain the gene locus name

      data.each do |item|
        (0..item.length-1).each do |k|
          interactions_array.push(item[k].scan(/A[Tt]\d[Gg]\d\d\d\d\d/)) #find the agi code in the text, retrieve it and add it to the array
        end
      end
    end
    
    interactions_array = interactions_array.flatten.uniq  #unnest the arrray, delete repetitions and interactions with itself
    interactions_array.map!(&:upcase) #make all the codes uppercase so that I can compare them latter
    interactions_array = interactions_array- [my_gene.upcase] #dont include my own gene as an interaction
    #puts "interactions_array created"
    
    return interactions_array
    
  end
end

:get_interactions

# Get interaction information

#

#

# ANNOTATION

In [221]:
ppi_member=Array.new

ppi.each do |item|
  ppi_member.push('new')
  ppi_member[-1]=Gene.new(
    :gene_id => item[0],
    #:prot_id => get_protID(item[0]),
    #:kegg => get_KEGG(item[0]),
    #:go => get_GO(item[0]),
    :interactions => item[1]
     )
  puts ppi_member[-1].get_info
  puts ""
end

Gene's ID : AT5G19120
Proteins's UniProt name: []
KEGG's ID and pathway name: []
GO's ID and term name: []
Protein interactions: ["AT3G46090"]


Gene's ID : AT2G13360
Proteins's UniProt name: []
KEGG's ID and pathway name: []
GO's ID and term name: []
Protein interactions: ["AT3G46090", "AT2G01950"]


Gene's ID : AT5G04140
Proteins's UniProt name: []
KEGG's ID and pathway name: []
GO's ID and term name: []
Protein interactions: ["AT1G78300"]


Gene's ID : AT1G03130
Proteins's UniProt name: []
KEGG's ID and pathway name: []
GO's ID and term name: []
Protein interactions: ["AT2G23290", "AT4G29930", "AT5G32440", "AT3G08530", "AT4G19700", "AT3G47620", "AT5G37670", "AT5G61010", "AT3G02150", "AT3G48150"]


Gene's ID : AT4G17460
Proteins's UniProt name: []
KEGG's ID and pathway name: []
GO's ID and term name: []
Protein interactions: ["AT3G07220", "AT3G15030", "AT5G47370", "AT2G45680", "AT1G60250", "AT3G02150", "AT3G47620"]


Gene's ID : AT4G22890
Proteins's UniProt name: []
KEGG's ID and pat

{"AT5G19120"=>["AT3G46090"], "AT2G13360"=>["AT3G46090", "AT2G01950"], "AT5G04140"=>["AT1G78300"], "AT1G03130"=>["AT2G23290", "AT4G29930", "AT5G32440", "AT3G08530", "AT4G19700", "AT3G47620", "AT5G37670", "AT5G61010", "AT3G02150", "AT3G48150"], "AT4G17460"=>["AT3G07220", "AT3G15030", "AT5G47370", "AT2G45680", "AT1G60250", "AT3G02150", "AT3G47620"], "AT4G22890"=>["AT4G37680", "AT1G17280", "AT1G29060", "AT5G59650", "AT4G20790", "AT2G22425", "AT3G12180", "AT1G20020", "AT4G02770"], "AT1G11850"=>["AT4G26630"], "AT3G01500"=>["AT4G16620", "AT2G28960", "AT5G42980"], "AT4G33010"=>["AT1G78300"], "AT2G17880"=>["AT3G48150", "AT3G49580", "AT4G17680"], "AT4G21210"=>["AT5G02850"], "AT5G14740"=>["AT5G42980"], "AT2G39730"=>["AT5G42980"], "AT1G20020"=>["AT4G22890"], "AT2G34420"=>["AT3G47620"], "AT3G55800"=>["AT5G42980"], "AT5G15850"=>["AT1G42050", "AT3G02150", "AT3G07220", "AT4G35550", "AT2G46340"], "AT5G17300"=>["AT3G15030", "AT1G60250", "AT2G25000", "AT2G45680", "AT3G02150", "AT3G47620", "AT4G35550", "A

[]

# Create netwok

In [159]:
## first level interactions
int_L1=Array.new
annotate_int_L1=Array.new

genes_list.each do |gene|
   interactions=get_interactions(gene,genes_list) #get interactions from web accessing function
   if interactions != [] # if there are interacions
     annotate_int_L1.push([gene,interactions]) # add that interaction as [my_gene, [array of interactions]] 
     int_L1.push([gene,interactions].flatten) # add that interaction as [my_gene,interaction1,interaction2, ...] 
   end
 end

["AT4G27030", "AT5G54270", "AT1G21400", "AT5G19120", "AT2G13360", "AT4G05180", "AT1G22690", "AT2G45170", "AT4G09650", "AT5G55620", "AT1G31330", "AT1G80440", "AT3G28270", "AT4G12800", "AT5G04140", "AT5G64040", "AT1G29660", "AT1G15820", "AT1G64500", "AT1G03130", "AT2G20670", "AT4G17090", "AT1G23740", "AT2G46340", "AT4G17460", "AT1G07010", "AT1G32220", "AT2G03750", "AT3G47470", "AT4G12830", "AT5G05690", "AT5G64410", "AT2G21330", "AT1G15980", "AT2G01890", "AT1G12250", "AT2G21210", "AT4G28660", "AT1G52230", "AT3G18890", "AT4G22890", "AT1G11850", "AT1G37130", "AT2G04039", "AT3G48420", "AT4G19170", "AT5G07020", "AT5G65010", "AT3G01500", "AT1G25230", "AT2G34430", "AT1G18060", "AT2G30600", "AT4G33010", "AT1G65490", "AT3G21670", "AT4G27260", "AT1G12780", "AT1G55480", "AT2G17880", "AT3G49160", "AT4G21210", "AT5G14740", "AT4G37980", "AT1G29395", "AT2G39730", "AT1G20020", "AT2G34420", "AT4G37800", "AT1G70820", "AT3G26740", "AT4G28780", "AT1G12900", "AT1G64680", "AT2G25200", "AT3G55800", "AT4G21280"

In [161]:
puts int_L1

[["AT5G19120", "AT3G46090"], ["AT2G13360", "AT3G14415", "AT3G46090", "AT2G01950"], ["AT4G05180", "AT4G25200"], ["AT4G12800", "AT2G46820"], ["AT5G04140", "AT4G37930", "AT1G78300"], ["AT1G03130", "AT2G23290", "AT4G08150", "AT4G29930", "AT4G32190", "AT5G32440", "AT3G08530", "AT4G19700", "AT3G47620", "AT5G37670", "AT5G61010", "AT3G02150", "AT3G48150", "AT1G52220"], ["AT4G17460", "AT1G20700", "AT3G07220", "AT1G70920", "AT3G15030", "AT2G44910", "AT4G16780", "AT5G12980", "AT3G60390", "AT5G47370", "AT2G22800", "AT2G45680", "AT1G60250", "AT2G33835", "AT3G02150", "AT1G30490", "AT3G47620", "AT4G39780", "AT4G37790"], ["AT1G07010", "AT4G17030"], ["AT5G05690", "AT4G03820"], ["AT2G01890", "AT4G23640"], ["AT4G22890", "AT1G18520", "AT5G25752", "AT4G37680", "AT1G17280", "AT1G29060", "AT5G59650", "AT4G20790", "AT1G21240", "AT2G22425", "AT3G12180", "AT4G38690", "AT2G05620", "AT1G60950", "AT5G66190", "AT1G20020", "AT4G02770"], ["AT1G11850", "AT4G26630", "AT3G16310", "AT5G22290"], ["AT1G37130", "AT5G38640"]

In [162]:
## second level interactions
int_L2=Array.new
int_L1.each do |gene|  # takes something like [my_gene, interaction1, interaction2, ...]
  puts ""
  puts "gene #{gene}"
  gene[1..-1].each do |interaction| #takes each item of [interaction1, interaction2, ...]
    puts "interactions #{gene[1..-1]}"
    int_list=[]
    int_L1.each do |item| # per set of [my_gene, [interaction1, interaction2, ...]]
      puts "item #{item}"
      if item[1..-1].include? interaction # if [interaction1, interaction2, ...] includes my_gene
        puts "#{item[1..-1]} includes #{interaction}, I add #{item[0]}"
        int_list.push([item[0]]) #add the my_gene to the list
      end
    end
    int_list.flatten!.uniq! # unnest arrays and remove duplicated genes
    if int_list.length > 1 #there is interaction with 2 genes and therefore forms a network
      int_L2.push([interaction,int_list].flatten)
    end
  end
end




gene ["AT5G19120", "AT3G46090"]
interactions ["AT3G46090"]
item ["AT5G19120", "AT3G46090"]
["AT3G46090"] includes AT3G46090, I add AT5G19120
item ["AT2G13360", "AT3G14415", "AT3G46090", "AT2G01950"]
["AT3G14415", "AT3G46090", "AT2G01950"] includes AT3G46090, I add AT2G13360
item ["AT4G05180", "AT4G25200"]
item ["AT4G12800", "AT2G46820"]
item ["AT5G04140", "AT4G37930", "AT1G78300"]
item ["AT1G03130", "AT2G23290", "AT4G08150", "AT4G29930", "AT4G32190", "AT5G32440", "AT3G08530", "AT4G19700", "AT3G47620", "AT5G37670", "AT5G61010", "AT3G02150", "AT3G48150", "AT1G52220"]
item ["AT4G17460", "AT1G20700", "AT3G07220", "AT1G70920", "AT3G15030", "AT2G44910", "AT4G16780", "AT5G12980", "AT3G60390", "AT5G47370", "AT2G22800", "AT2G45680", "AT1G60250", "AT2G33835", "AT3G02150", "AT1G30490", "AT3G47620", "AT4G39780", "AT4G37790"]
item ["AT1G07010", "AT4G17030"]
item ["AT5G05690", "AT4G03820"]
item ["AT2G01890", "AT4G23640"]
item ["AT4G22890", "AT1G18520", "AT5G25752", "AT4G37680", "AT1G17280", "AT1G29

item ["AT4G12800", "AT2G46820"]
item ["AT5G04140", "AT4G37930", "AT1G78300"]
item ["AT1G03130", "AT2G23290", "AT4G08150", "AT4G29930", "AT4G32190", "AT5G32440", "AT3G08530", "AT4G19700", "AT3G47620", "AT5G37670", "AT5G61010", "AT3G02150", "AT3G48150", "AT1G52220"]
item ["AT4G17460", "AT1G20700", "AT3G07220", "AT1G70920", "AT3G15030", "AT2G44910", "AT4G16780", "AT5G12980", "AT3G60390", "AT5G47370", "AT2G22800", "AT2G45680", "AT1G60250", "AT2G33835", "AT3G02150", "AT1G30490", "AT3G47620", "AT4G39780", "AT4G37790"]
item ["AT1G07010", "AT4G17030"]
item ["AT5G05690", "AT4G03820"]
item ["AT2G01890", "AT4G23640"]
item ["AT4G22890", "AT1G18520", "AT5G25752", "AT4G37680", "AT1G17280", "AT1G29060", "AT5G59650", "AT4G20790", "AT1G21240", "AT2G22425", "AT3G12180", "AT4G38690", "AT2G05620", "AT1G60950", "AT5G66190", "AT1G20020", "AT4G02770"]
item ["AT1G11850", "AT4G26630", "AT3G16310", "AT5G22290"]
item ["AT1G37130", "AT5G38640"]
item ["AT4G19170", "AT5G17790", "AT5G56760"]
item ["AT3G01500", "AT4G

item ["AT1G07010", "AT4G17030"]
item ["AT5G05690", "AT4G03820"]
item ["AT2G01890", "AT4G23640"]
item ["AT4G22890", "AT1G18520", "AT5G25752", "AT4G37680", "AT1G17280", "AT1G29060", "AT5G59650", "AT4G20790", "AT1G21240", "AT2G22425", "AT3G12180", "AT4G38690", "AT2G05620", "AT1G60950", "AT5G66190", "AT1G20020", "AT4G02770"]
item ["AT1G11850", "AT4G26630", "AT3G16310", "AT5G22290"]
item ["AT1G37130", "AT5G38640"]
item ["AT4G19170", "AT5G17790", "AT5G56760"]
item ["AT3G01500", "AT4G30190", "AT3G59780", "AT4G16620", "AT5G25260", "AT1G05640", "AT2G01680", "AT5G19410", "AT2G28960", "AT5G38660", "AT5G42980", "AT5G67480"]
item ["AT2G30600", nil]
item ["AT4G33010", "AT1G78300"]
item ["AT1G12780", "AT4G23920"]
item ["AT2G17880", "AT3G48150", "AT3G49580", "AT4G17680"]
item ["AT3G49160", "AT3G15420", "AT2G47710", "AT2G42910", "AT2G01760", "AT4G16143", "AT1G71050", "AT2G36930"]
item ["AT4G21210", "AT5G02850"]
item ["AT5G14740", "AT5G42980", "AT1G23730", "AT1G19580", "AT5G66510", "AT5G63510", "AT3G486

item ["AT1G37130", "AT5G38640"]
item ["AT4G19170", "AT5G17790", "AT5G56760"]
item ["AT3G01500", "AT4G30190", "AT3G59780", "AT4G16620", "AT5G25260", "AT1G05640", "AT2G01680", "AT5G19410", "AT2G28960", "AT5G38660", "AT5G42980", "AT5G67480"]
item ["AT2G30600", nil]
item ["AT4G33010", "AT1G78300"]
item ["AT1G12780", "AT4G23920"]
item ["AT2G17880", "AT3G48150", "AT3G49580", "AT4G17680"]
item ["AT3G49160", "AT3G15420", "AT2G47710", "AT2G42910", "AT2G01760", "AT4G16143", "AT1G71050", "AT2G36930"]
item ["AT4G21210", "AT5G02850"]
item ["AT5G14740", "AT5G42980", "AT1G23730", "AT1G19580", "AT5G66510", "AT5G63510", "AT3G48680", "AT3G06310", "AT2G33220", "AT2G02510", "AT1G49140", "AT3G18410", "AT2G47690", "AT1G79010", "AT3G08610", "AT5G47890", "AT5G52840", "AT3G12260", "AT2G20360", "AT3G03100", "AT1G76200", "AT2G02050", "AT5G47570", "AT5G37510", "AT5G67590", "AT5G11770", "AT5G08530", "AT4G02580", "AT2G07785", "AT2G07786", "AT4G16450", "AT4G20150", "AT2G42210", "AT4G28510", "AT1G03860", "AT5G40770",

item ["AT4G33010", "AT1G78300"]
item ["AT1G12780", "AT4G23920"]
item ["AT2G17880", "AT3G48150", "AT3G49580", "AT4G17680"]
item ["AT3G49160", "AT3G15420", "AT2G47710", "AT2G42910", "AT2G01760", "AT4G16143", "AT1G71050", "AT2G36930"]
item ["AT4G21210", "AT5G02850"]
item ["AT5G14740", "AT5G42980", "AT1G23730", "AT1G19580", "AT5G66510", "AT5G63510", "AT3G48680", "AT3G06310", "AT2G33220", "AT2G02510", "AT1G49140", "AT3G18410", "AT2G47690", "AT1G79010", "AT3G08610", "AT5G47890", "AT5G52840", "AT3G12260", "AT2G20360", "AT3G03100", "AT1G76200", "AT2G02050", "AT5G47570", "AT5G37510", "AT5G67590", "AT5G11770", "AT5G08530", "AT4G02580", "AT2G07785", "AT2G07786", "AT4G16450", "AT4G20150", "AT2G42210", "AT4G28510", "AT1G03860", "AT5G40770", "AT3G27280", "AT2G20530", "AT5G44140", "AT1G68680", "AT4G00585", "AT2G31490", "AT2G42310", "AT2G27730"]
item ["AT1G29395", nil]
item ["AT2G39730", "AT5G42980"]
item ["AT1G20020", "AT4G22890"]
item ["AT2G34420", "AT3G47620"]
item ["AT4G28780", "AT4G35580", "AT1G2

...

item ["AT1G29920", "AT3G47620", "AT1G74890", "AT1G29910"]
item ["AT4G02770", "AT2G23290", "AT3G24140", "AT4G29930", "AT1G09660", "AT1G09670", "AT5G32440", "AT3G08530", "AT4G17680", "AT3G06940", "AT4G19700", "AT4G30080", "AT3G47620", "AT5G37670", "AT5G24660", "AT5G61010", "AT1G15670", "AT1G01260", "AT3G61150", "AT3G49580", "AT3G48150", "AT4G22890"]
["AT2G23290", "AT3G24140", "AT4G29930", "AT1G09660", "AT1G09670", "AT5G32440", "AT3G08530", "AT4G17680", "AT3G06940", "AT4G19700", "AT4G30080", "AT3G47620", "AT5G37670", "AT5G24660", "AT5G61010", "AT1G15670", "AT1G01260", "AT3G61150", "AT3G49580", "AT3G48150", "AT4G22890"] includes AT1G09660, I add AT4G02770
item ["AT4G39710", "AT4G13530", "AT4G03280"]
item ["AT5G48300", "AT2G21590", "AT3G09250"]
item ["AT4G09010", "AT3G53900"]
item ["AT5G01530", "AT2G01950"]
interactions ["AT2G23290", "AT3G24140", "AT4G29930", "AT1G09660", "AT1G09670", "AT5G32440", "AT3G08530", "AT4G17680", "AT3G06940", "AT4G19700", "AT4G30080", "AT3G47620", "AT5G37670", "AT

item ["AT1G29920", "AT3G47620", "AT1G74890", "AT1G29910"]
item ["AT4G02770", "AT2G23290", "AT3G24140", "AT4G29930", "AT1G09660", "AT1G09670", "AT5G32440", "AT3G08530", "AT4G17680", "AT3G06940", "AT4G19700", "AT4G30080", "AT3G47620", "AT5G37670", "AT5G24660", "AT5G61010", "AT1G15670", "AT1G01260", "AT3G61150", "AT3G49580", "AT3G48150", "AT4G22890"]
["AT2G23290", "AT3G24140", "AT4G29930", "AT1G09660", "AT1G09670", "AT5G32440", "AT3G08530", "AT4G17680", "AT3G06940", "AT4G19700", "AT4G30080", "AT3G47620", "AT5G37670", "AT5G24660", "AT5G61010", "AT1G15670", "AT1G01260", "AT3G61150", "AT3G49580", "AT3G48150", "AT4G22890"] includes AT1G09670, I add AT4G02770
item ["AT4G39710", "AT4G13530", "AT4G03280"]
item ["AT5G48300", "AT2G21590", "AT3G09250"]
item ["AT4G09010", "AT3G53900"]
item ["AT5G01530", "AT2G01950"]
interactions ["AT2G23290", "AT3G24140", "AT4G29930", "AT1G09660", "AT1G09670", "AT5G32440", "AT3G08530", "AT4G17680", "AT3G06940", "AT4G19700", "AT4G30080", "AT3G47620", "AT5G37670", "AT

item ["AT4G04640", "AT5G44520"]
item ["AT5G38990", "AT1G17280", "AT3G12180", "AT2G26180", "AT3G45290", "AT5G11890", "AT5G06320", "AT1G60940", "AT2G28250", "AT4G30850", "AT4G20790", "AT5G59650", "AT1G29060", "AT2G32380", "AT1G47640", "AT1G07860"]
item ["AT1G29920", "AT3G47620", "AT1G74890", "AT1G29910"]
item ["AT4G02770", "AT2G23290", "AT3G24140", "AT4G29930", "AT1G09660", "AT1G09670", "AT5G32440", "AT3G08530", "AT4G17680", "AT3G06940", "AT4G19700", "AT4G30080", "AT3G47620", "AT5G37670", "AT5G24660", "AT5G61010", "AT1G15670", "AT1G01260", "AT3G61150", "AT3G49580", "AT3G48150", "AT4G22890"]
["AT2G23290", "AT3G24140", "AT4G29930", "AT1G09660", "AT1G09670", "AT5G32440", "AT3G08530", "AT4G17680", "AT3G06940", "AT4G19700", "AT4G30080", "AT3G47620", "AT5G37670", "AT5G24660", "AT5G61010", "AT1G15670", "AT1G01260", "AT3G61150", "AT3G49580", "AT3G48150", "AT4G22890"] includes AT5G32440, I add AT4G02770
item ["AT4G39710", "AT4G13530", "AT4G03280"]
item ["AT5G48300", "AT2G21590", "AT3G09250"]
item

item ["AT3G50820", "AT5G42980"]
item ["AT1G13930", "AT4G26630"]
item ["AT4G37610", "AT1G61660", "AT1G26830"]
item ["AT1G76100", "AT5G02850"]
item ["AT3G61260", "AT3G17210", "AT3G48040", "AT2G26180", "AT3G12180", "AT3G55440", "AT1G10470"]
item ["AT4G04640", "AT5G44520"]
item ["AT5G38990", "AT1G17280", "AT3G12180", "AT2G26180", "AT3G45290", "AT5G11890", "AT5G06320", "AT1G60940", "AT2G28250", "AT4G30850", "AT4G20790", "AT5G59650", "AT1G29060", "AT2G32380", "AT1G47640", "AT1G07860"]
item ["AT1G29920", "AT3G47620", "AT1G74890", "AT1G29910"]
item ["AT4G02770", "AT2G23290", "AT3G24140", "AT4G29930", "AT1G09660", "AT1G09670", "AT5G32440", "AT3G08530", "AT4G17680", "AT3G06940", "AT4G19700", "AT4G30080", "AT3G47620", "AT5G37670", "AT5G24660", "AT5G61010", "AT1G15670", "AT1G01260", "AT3G61150", "AT3G49580", "AT3G48150", "AT4G22890"]
["AT2G23290", "AT3G24140", "AT4G29930", "AT1G09660", "AT1G09670", "AT5G32440", "AT3G08530", "AT4G17680", "AT3G06940", "AT4G19700", "AT4G30080", "AT3G47620", "AT5G3767

Interrupt: 

In [152]:
## merge and make that array a local variable so it can be called later on
$all=[int_L1,int_L2].flatten!(1)

[["AT3G46090", "AT5G19120"], ["AT2G01950", "AT2G13360", "AT3G14415", "AT3G46090"], ["AT4G05180", "AT4G25200"], ["AT2G46820", "AT4G12800"], ["AT1G78300", "AT4G37930", "AT5G04140"], ["AT1G03130", "AT1G52220", "AT2G23290", "AT3G02150", "AT3G08530", "AT3G47620", "AT3G48150", "AT4G08150", "AT4G19700", "AT4G29930", "AT4G32190", "AT5G32440", "AT5G37670", "AT5G61010"], ["AT1G20700", "AT1G30490", "AT1G60250", "AT1G70920", "AT2G22800", "AT2G33835", "AT2G44910", "AT2G45680", "AT3G02150", "AT3G07220", "AT3G15030", "AT3G47620", "AT3G60390", "AT4G16780", "AT4G17460", "AT4G37790", "AT4G39780", "AT5G12980", "AT5G47370"], ["AT1G07010", "AT4G17030"], ["AT4G03820", "AT5G05690"], ["AT2G01890", "AT4G23640"], ["AT1G17280", "AT1G18520", "AT1G20020", "AT1G21240", "AT1G29060", "AT1G60950", "AT2G05620", "AT2G22425", "AT3G12180", "AT4G02770", "AT4G20790", "AT4G22890", "AT4G37680", "AT4G38690", "AT5G25752", "AT5G59650", "AT5G66190"], ["AT1G11850", "AT3G16310", "AT4G26630", "AT5G22290"], ["AT1G37130", "AT5G38640"]

In [151]:
int_L2.length

107

In [153]:
networks=Array.new
$all.each do |item| # get one array of interactions, for example [gene1 , gene2, gene3]
  net=Array.new
  
  item.each do |elem| # get one of those genes, for example gene1
    (0..$all.length-1).each do |i| #iterate over the arrays again (like all[0] = [gene1 , gene2, gene3])
      if $all[i].any? elem #if the array contains that gene (for example gene1 is in all[0])
        net.push($all[i].flatten) # include that array of interactions to my new array 
      end
    end
  end
  net.flatten!.uniq!.sort! #make the array: unnested, no duplicates and sorted 
  if net.length > 2 
    networks.push(net)
  end
end

networks.uniq! #remove duplicated networks


[["AT2G01950", "AT2G13360", "AT3G14415", "AT3G46090", "AT5G19120"], ["AT1G10090", "AT2G01950", "AT2G13360", "AT2G47930", "AT3G14415", "AT3G15710", "AT3G46090", "AT5G01530", "AT5G19120"], ["AT1G09530", "AT1G09570", "AT1G78300", "AT2G18790", "AT2G20180", "AT2G25930", "AT2G40080", "AT2G43010", "AT3G46640", "AT4G33010", "AT4G37930", "AT5G04140"], ["AT1G01260", "AT1G03040", "AT1G03130", "AT1G03150", "AT1G04990", "AT1G05710", "AT1G08465", "AT1G08970", "AT1G09660", "AT1G09670", "AT1G10586", "AT1G14920", "AT1G15670", "AT1G20700", "AT1G20910", "AT1G22920", "AT1G26780", "AT1G29910", "AT1G29920", "AT1G30490", "AT1G36060", "AT1G42050", "AT1G48195", "AT1G48500", "AT1G52220", "AT1G53090", "AT1G60250", "AT1G60380", "AT1G62085", "AT1G63820", "AT1G69180", "AT1G70920", "AT1G71130", "AT1G72310", "AT1G72570", "AT1G74890", "AT1G74930", "AT1G75520", "AT1G76510", "AT2G01570", "AT2G05330", "AT2G14880", "AT2G17880", "AT2G18120", "AT2G22800", "AT2G23290", "AT2G23420", "AT2G25000", "AT2G27930", "AT2G30432", "AT2

In [154]:
networks.length


56

In [1]:
a= ["AT1G78300", "AT4G37930", ["AT1G78300","AT5G04140"]]

["AT1G78300", "AT4G37930", ["AT1G78300", "AT5G04140"]]

In [2]:
a.flatten!.uniq!

["AT1G78300", "AT4G37930", "AT5G04140"]

In [3]:
a

["AT1G78300", "AT4G37930", "AT5G04140"]

In [4]:
a[1..-1]

["AT4G37930", "AT5G04140"]

89

[]

In [151]:
c={"a"=>[1,2,3],"b"=>[4,2,5],"c"=>[4,7,8]}


{"a"=>[1, 2, 3], "b"=>[4, 2, 5], "c"=>[4, 7, 8]}